# Projeto Big Data
## Alunos: Keiya Nishio e Pedro Dannecker

Biblioteca utilizada para a coleta de dados: https://github.com/swar/nba_api/tree/master<br>
Projeto realizado no Google Colab

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
import pyspark.sql.functions as func
import plotly.express as px

In [2]:
!pip install nba_api
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']

In [3]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612738,BOS,Boston Celtics,0022400218,2024-11-13,BOS @ BKN,W,241,139,...,0.708,12,33,45,35,7,6,7,17,25.0
1,22024,1610612738,BOS,Boston Celtics,0022400001,2024-11-12,BOS vs. ATL,L,239,116,...,0.815,6,28,34,28,7,5,20,12,-1.0
2,22024,1610612738,BOS,Boston Celtics,0022400202,2024-11-10,BOS @ MIL,W,240,113,...,0.889,8,34,42,27,5,2,12,16,6.0
3,22024,1610612738,BOS,Boston Celtics,0022400187,2024-11-08,BOS vs. BKN,W,266,108,...,0.842,4,35,39,23,7,9,13,15,4.0
4,22024,1610612738,BOS,Boston Celtics,0022400172,2024-11-06,BOS vs. GSW,L,240,112,...,0.810,13,34,47,22,5,3,12,18,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,21983,1610612738,BOS,Boston Celtics,0028300052,1983-11-05,BOS @ WAS,W,240,120,...,0.778,5,28,33,36,7,7,16,26,NaN
3931,21983,1610612738,BOS,Boston Celtics,0028300043,1983-11-04,BOS vs. IND,W,240,121,...,0.841,11,34,45,28,8,5,22,26,NaN
3932,21983,1610612738,BOS,Boston Celtics,0028300033,1983-11-02,BOS vs. MIL,W,240,119,...,0.816,14,24,38,26,7,7,18,18,NaN
3933,21983,1610612738,BOS,Boston Celtics,0028300019,1983-10-29,BOS @ CLE,W,240,108,...,0.690,13,31,44,25,12,7,13,22,NaN


In [4]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [5]:
games.MATCHUP

,MATCHUP
0,BOS @ BKN
1,BOS vs. ATL
2,BOS @ MIL
3,BOS vs. BKN
4,BOS vs. GSW
...,...
3930,BOS @ WAS
3931,BOS vs. IND
3932,BOS vs. MIL
3933,BOS @ CLE


## Tratamento da base

In [6]:
new_Column = [1 if "@" in game else 0 for game in games.MATCHUP]
new_Column
games["Home Court"] = new_Column

In [7]:
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Home Court
0,22024,1610612738,BOS,Boston Celtics,0022400218,2024-11-13,BOS @ BKN,W,241,139,...,12,33,45,35,7,6,7,17,25.0,1
1,22024,1610612738,BOS,Boston Celtics,0022400001,2024-11-12,BOS vs. ATL,L,239,116,...,6,28,34,28,7,5,20,12,-1.0,0
2,22024,1610612738,BOS,Boston Celtics,0022400202,2024-11-10,BOS @ MIL,W,240,113,...,8,34,42,27,5,2,12,16,6.0,1
3,22024,1610612738,BOS,Boston Celtics,0022400187,2024-11-08,BOS vs. BKN,W,266,108,...,4,35,39,23,7,9,13,15,4.0,0
4,22024,1610612738,BOS,Boston Celtics,0022400172,2024-11-06,BOS vs. GSW,L,240,112,...,13,34,47,22,5,3,12,18,-6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,21983,1610612738,BOS,Boston Celtics,0028300052,1983-11-05,BOS @ WAS,W,240,120,...,5,28,33,36,7,7,16,26,NaN,1
3931,21983,1610612738,BOS,Boston Celtics,0028300043,1983-11-04,BOS vs. IND,W,240,121,...,11,34,45,28,8,5,22,26,NaN,0
3932,21983,1610612738,BOS,Boston Celtics,0028300033,1983-11-02,BOS vs. MIL,W,240,119,...,14,24,38,26,7,7,18,18,NaN,0
3933,21983,1610612738,BOS,Boston Celtics,0028300019,1983-10-29,BOS @ CLE,W,240,108,...,13,31,44,25,12,7,13,22,NaN,1


**Criar sessão spark**

In [8]:
# Create a SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# **Análises**

In [9]:
games_df = spark.createDataFrame(games)
games_df.show()


+---------+----------+-----------------+--------------+----------+----------+-----------+---+---+---+---+---+------+----+----+-------+---+---+------+----+----+---+---+---+---+---+---+----------+----------+
|SEASON_ID|   TEAM_ID|TEAM_ABBREVIATION|     TEAM_NAME|   GAME_ID| GAME_DATE|    MATCHUP| WL|MIN|PTS|FGM|FGA|FG_PCT|FG3M|FG3A|FG3_PCT|FTM|FTA|FT_PCT|OREB|DREB|REB|AST|STL|BLK|TOV| PF|PLUS_MINUS|Home Court|
+---------+----------+-----------------+--------------+----------+----------+-----------+---+---+---+---+---+------+----+----+-------+---+---+------+----+----+---+---+---+---+---+---+----------+----------+
|    22024|1610612738|              BOS|Boston Celtics|0022400218|2024-11-13|  BOS @ BKN|  W|241|139| 50| 93| 0.538|  22|  46|  0.478| 17| 24| 0.708|  12|  33| 45| 35|  7|  6|  7| 17|      25.0|         1|
|    22024|1610612738|              BOS|Boston Celtics|0022400001|2024-11-12|BOS vs. ATL|  L|239|116| 38| 75| 0.507|  18|  45|    0.4| 22| 27| 0.815|   6|  28| 34| 28|  7|  5| 

In [10]:
games_df = games_df[games_df["PTS"] > 2]
games_df = games_df.withColumn(
    "WL",
    func.when(games_df["PTS"] == 36, "W").otherwise(games_df["WL"])
)

### Ano x Pontuação (PTS)

In [11]:
games_df.select("PTS").describe().show()

+-------+------------------+
|summary|               PTS|
+-------+------------------+
|  count|              3934|
|   mean|103.62302999491611|
| stddev|14.125081005799823|
|    min|                36|
|    max|               157|
+-------+------------------+



In [12]:
avg_points = games_df \
    .withColumn("YEAR", func.year(func.col("GAME_DATE"))) \
    .groupBy("YEAR") \
    .agg(func.avg("PTS").alias("AVG_PTS")) \
    .orderBy("YEAR")


# Converte para pandas
avg_points_pandas = avg_points.toPandas()
avg_points_pandas.columns = ["Ano", "Média de Pontos Anual"]
grafico = px.bar(avg_points_pandas,x="Ano",y="Média de Pontos Anual")

grafico.update_layout(
    xaxis=dict(
        tickmode='linear',
        tickangle=45
    ),
    title="Média de Pontos por Ano",
    xaxis_title="Ano",
    yaxis_title="Média de Pontos Anual"
)


In [13]:
sum_points_per_year = games_df \
    .withColumn("YEAR", func.year(func.col("GAME_DATE"))) \
    .groupBy("YEAR") \
    .agg(func.sum("PTS").alias("SUM_PTS")) \
    .orderBy("YEAR")


sum_points_pandas = sum_points_per_year.toPandas()
sum_points_pandas.columns = ["Ano", "Soma de Pontos Anual"]
grafico = px.bar(sum_points_pandas,x="Ano",y="Soma de Pontos Anual")

grafico.update_layout(
    xaxis=dict(
        tickmode='linear',
        tickangle=45
    ),
    title="Soma de Pontos por Ano",
    xaxis_title="Ano",
    yaxis_title="Soma de Pontos Anual"
)



### Vitórias e Derrotas (WL) x Pontuação (PTS)

In [17]:
games_df.groupBy("WL").count().show()


+---+-----+
| WL|count|
+---+-----+
|  L| 1699|
|  W| 2235|
+---+-----+



In [20]:
games_wl = games_df.groupBy("WL").agg(func.avg("PTS"))

df_wl = games_wl.toPandas()
df_wl.columns = ["Vitória e Derrota", "Média de Pontuação"]
df_wl["Vitória e Derrota"] = df_wl["Vitória e Derrota"].replace({"L": "Derrota", "W": "Vitória"})
grafico = px.bar(df_wl, x="Vitória e Derrota", y="Média de Pontuação")

grafico.update_layout(
    width=600,
    height=400,
    title="Média de Pontos por Vitórias e Derrotas",
    xaxis_title="Ano",
    yaxis_title="Média de Pontos"
)


### Número de cestas realizadas (FGM) x Pontuação (PTS)

In [45]:
games_fgm = games_df.groupBy("FGM").agg(func.avg("PTS"))

df_fgm = games_fgm.toPandas()
df_fgm.columns = ["Número de cestas", "Média de Pontuação"]
df_fgm = df_fgm.sort_values(by="Média de Pontuação", ascending=False).head(50)

grafico = px.bar(df_fgm, x="Número de cestas", y="Média de Pontuação")

grafico.update_layout(
    xaxis=dict(
        tickmode='linear',
        tickangle=45
    ),
    title="Média de Pontos por Número de cestas",
    xaxis_title="Número de cestas",
    yaxis_title="Média de Pontos"
)


### Número de tentativas de arremesso (FGA) X Pontuação (PTS)

In [47]:
games_fga = games_df.groupBy("FGA").agg(func.avg("PTS"))

df_fga = games_fga.toPandas()
df_fga.columns = ["Número de tentativas", "Média de Pontos"]
df_fga = df_fga.sort_values(by="Média de Pontos", ascending=False).head(50)
grafico = px.bar(df_fga, x="Número de tentativas", y="Média de Pontos")
grafico.update_layout(
    xaxis=dict(
        tickmode='linear',
        tickangle=45
    ),
    title="Média de Pontos por Número de tentativas",
    xaxis_title="Número de tentativas",
    yaxis_title="Média de Pontos"
)



### Número de cestas realizadas de 3 pontos (FG3M) x Pontuação (PTS)

In [35]:
games_fg3m = games_df.groupBy("FG3M").agg(func.avg("PTS"))

df_fg3m = games_fg3m.toPandas()
df_fg3m.columns = ["Número de cestas", "Média de Pontos"]
df_fg3m = df_fg3m.sort_values(by="Média de Pontos", ascending=False).head(50)
grafico = px.bar(df_fg3m, x="Número de cestas", y="Média de Pontos")
grafico.update_layout(
    xaxis=dict(
        tickmode='linear',
        tickangle=0
    ),
    title="Média de Pontos por Número de cestas (3 pontos)",
    xaxis_title="Número de cestas (3 pontos)",
    yaxis_title="Média de Pontos"
)


### Bloqueio (BLK) x Pontuação (PTS)

In [34]:
games_blk = games_df.groupBy("BLK").agg(func.avg("PTS"))

df_blk = games_blk.toPandas()
df_blk.columns = ["Número de bloqueios", "Média de Pontos"]
df_blk = df_blk.sort_values(by="Média de Pontos", ascending=False).head(50)
grafico = px.bar(df_blk, x="Número de bloqueios", y="Média de Pontos")
grafico.update_layout(
    xaxis=dict(
        tickmode='linear',
        tickangle=0
    ),
    title="Média de Pontos por Número de bloqueios",
    xaxis_title="Número de bloqueios",
    yaxis_title="Média de Pontos"
)

# Machine Learning

**Feature Assembling e separando em Treino e Teste**

In [48]:
# Selecionar apenas as colunas relevantes e remover linhas com valores nulos
features = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
            'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
            'BLK', 'TOV', 'PF', 'PLUS_MINUS','Home Court']
target = 'PTS'

# Convert your pandas DataFrame 'games' to a Spark DataFrame
#games_df = spark.createDataFrame(games)

# # Defina os conjuntos de treino e teste com a condição de data
games_train_df = games_df.filter((col("GAME_DATE") <= "2016-04-14")) \
                   .select(*features, target) \
                   .dropna()
games_test_df = games_df.filter((col("GAME_DATE") > "2016-04-14")) \
                   .select(*features, target) \
                   .dropna()

games_df = games_df.select(*features, target).dropna()

# Criar o VectorAssembler para combinar as features em um vetor
assembler = VectorAssembler(inputCols=features, outputCol="features")
df_transform_train = assembler.transform(games_train_df)
df_transform_test = assembler.transform(games_test_df)

df_transform_train.show()
df_transform_test.show()


+---+---+---+------+----+----+-------+---+---+------+----+----+---+---+---+---+---+---+----------+----------+---+--------------------+
|MIN|FGM|FGA|FG_PCT|FG3M|FG3A|FG3_PCT|FTM|FTA|FT_PCT|OREB|DREB|REB|AST|STL|BLK|TOV| PF|PLUS_MINUS|Home Court|PTS|            features|
+---+---+---+------+----+----+-------+---+---+------+----+----+---+---+---+---+---+---+----------+----------+---+--------------------+
|240| 37| 86|  0.43|   5|  14|  0.357| 19| 22| 0.864|  10|  29| 39| 20|  7|  3|  7| 20|      10.0|         0| 98|[240.0,37.0,86.0,...|
|240| 34| 89| 0.382|   8|  24|  0.333| 24| 29| 0.828|  13|  33| 46| 17|  6|  1| 12| 17|     -14.0|         0|100|[240.0,34.0,89.0,...|
|241| 40| 90| 0.444|  11|  28|  0.393| 16| 24| 0.667|  10|  30| 40| 26| 10|  6| 15| 21|     -11.0|         1|107|[241.0,40.0,90.0,...|
|239| 44| 78| 0.564|   6|  14|  0.429| 30| 38| 0.789|   4|  30| 34| 35|  6|  9| 14| 23|      15.0|         0|124|[239.0,44.0,78.0,...|
|240| 36| 87| 0.414|  10|  34|  0.294| 22| 27| 0.815|  

**Random Forest**

In [49]:
# Inicializar o modelo Random Forest
rf = RandomForestRegressor(featuresCol="features", labelCol=target)

# # Treinar o modelo
model = rf.fit(df_transform_train)

# # Fazer previsões no conjunto de teste
predictions = model.transform(df_transform_test)


# # Mostrar algumas previsões
predictions.select("features", target, "prediction").show(5)

+--------------------+---+------------------+
|            features|PTS|        prediction|
+--------------------+---+------------------+
|[241.0,50.0,93.0,...|139| 115.7150464169008|
|[239.0,38.0,75.0,...|116|106.48784291885916|
|[240.0,35.0,86.0,...|113| 99.76698750912388|
|[266.0,39.0,90.0,...|108|103.46434941832528|
|[240.0,38.0,90.0,...|112|100.66138040667741|
+--------------------+---+------------------+
only showing top 5 rows



**Avaliando o modelo**

In [50]:
# # Avaliar o modelo
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Initialize RegressionEvaluator for MAE
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")

# Calculate MAE
mae = evaluator.evaluate(predictions)

print(f"Mean Absolute Error (MAE): {mae}")


Root Mean Squared Error (RMSE): 8.592561114700697
Mean Absolute Error (MAE): 6.774062958303354
